In [ ]:
import subprocess
import os
import glob

# Add cursor-agent to PATH
os.environ['PATH'] = f"{os.environ.get('HOME')}/.local/bin:" + os.environ.get('PATH', '')

# Find the cursor-agent installation directory
def find_cursor_agent_dir():
    """Find the cursor-agent installation directory"""
    home = os.environ.get('HOME', '')
    glob_pattern = f"{home}/.local/share/cursor-agent/versions/*/"
    dirs = glob.glob(glob_pattern)
    if dirs:
        # Sort and get the latest version
        return sorted(dirs)[-1]
    return None

cursor_agent_dir = find_cursor_agent_dir()
print(f"Found cursor-agent directory: {cursor_agent_dir}")

# Test cursor-agent
def test_cursor_agent_print():
    """Test cursor-agent with --print flag for non-interactive use"""
    try:
        # Try using cursor-agent command first
        result = subprocess.run(
            ["cursor-agent", "--print", "what does this app do?"],
            capture_output=True,
            text=True,
            timeout=180
        )
        if result.returncode == 0:
            print(f"Success! Return code: {result.returncode}")
            print(f"Output length: {len(result.stdout)} chars")
            print(f"\nOutput:\n{result.stdout}")
            if result.stderr:
                print(f"stderr: {result.stderr}")
            return result
        else:
            print(f"Failed with return code: {result.returncode}")
            print(f"stderr: {result.stderr}")
            
    except subprocess.TimeoutExpired:
        print("Timeout with cursor-agent command, trying direct Node path...")
        # Fallback to direct Node execution
        if cursor_agent_dir:
            node_bin = f"{cursor_agent_dir}/node"
            index_js = f"{cursor_agent_dir}/index.js"
            print(f"Trying: {node_bin} --use-system-ca {index_js} --print 'what does this app do?'")
            result = subprocess.run(
                [node_bin, "--use-system-ca", index_js, "--print", "what does this app do?"],
                capture_output=True,
                text=True,
                timeout=180
            )
            print(f"Success! Return code: {result.returncode}")
            print(f"\nOutput:\n{result.stdout}")
            if result.stderr:
                print(f"stderr: {result.stderr}")
            return result
        else:
            raise Exception("Could not find cursor-agent installation")
    return None

# Test the approach
print("Testing cursor-agent...")
result = test_cursor_agent_print()




TimeoutExpired: Command '['cursor-agent', '--print', 'what does this app do?']' timed out after 120 seconds

In [ ]:
def ask_cursor_agent(question, working_dir=None, output_format="text", timeout=180):
    """
    Ask a question to cursor-agent and get the response.
    
    Args:
        question: The question to ask
        working_dir: Optional working directory for the command
        output_format: Output format - "text", "json", or "stream-json" (default: "text")
        timeout: Timeout in seconds (default: 180)
    
    Returns:
        The response from cursor-agent
    """
    # First try using cursor-agent command
    cmd = ["cursor-agent", "--print", "--output-format", output_format, question]
    
    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            cwd=working_dir,
            timeout=timeout
        )
        
        # If successful, return the output
        if result.returncode == 0:
            return result.stdout
        else:
            # If failed, try direct Node execution
            print(f"[WARNING] cursor-agent command failed (code: {result.returncode}), trying direct Node path...")
            if result.stderr:
                print(f"stderr: {result.stderr}")
            
    except subprocess.TimeoutExpired:
        print(f"[WARNING] cursor-agent timed out, trying direct Node path...")
    
    # Fallback: use direct Node execution
    if cursor_agent_dir:
        node_bin = f"{cursor_agent_dir}/node"
        index_js = f"{cursor_agent_dir}/index.js"
        cmd = [node_bin, "--use-system-ca", index_js, "--print", "--output-format", output_format, question]
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            cwd=working_dir,
            timeout=timeout
        )
        
        if result.returncode == 0:
            return result.stdout
        else:
            error_msg = result.stderr if result.stderr else "Unknown error"
            raise Exception(f"Failed to get response from cursor-agent: {error_msg}")
    else:
        raise Exception("Could not find cursor-agent installation")


In [ ]:
# Now let's test asking a question with the new function
answer = ask_cursor_agent("what does this app do?")
print("\n" + "="*80)
print("ANSWER:")
print("="*80)
print(answer)

In [ ]:
# Test with a different question
answer2 = ask_cursor_agent("what are the key features of this project?")
print("\n" + "="*80)
print("ANSWER 2:")
print("="*80)
print(answer2)

'/Users/neo/Desktop/Uni/Sepideh/07.vs_extention\n'

In [ ]:
# Example: Ask about code structure
answer3 = ask_cursor_agent("how are the functions organized in cursor_cli.ipynb?")
print("\n" + "="*80)
print("ANSWER 3:")
print("="*80)
print(answer3)

In [ ]:
# You can also use JSON format for structured output
answer_json = ask_cursor_agent("list the main files in this project", output_format="json")
print("\n" + "="*80)
print("JSON ANSWER:")
print("="*80)
print(answer_json)

cursor_cli.ipynb
/Users/neo/Desktop/Uni/Sepideh/07.vs_extention
cursor_cli.ipynb
/Users/neo/Desktop/Uni/Sepideh/07.vs_extention
cursor_cli.ipynb
00.paper_reviews
cursor_cli.ipynb
/Users/neo/Desktop/Uni/Sepideh/07.vs_extention
cursor_cli.ipynb
/Users/neo/Desktop/Uni/Sepideh/07.vs_extention
cursor_cli.ipynb
00.paper_reviews



In [ ]:
# Create a simple wrapper function for easy use
def quick_ask(question):
    """Quick wrapper to ask cursor-agent a question"""
    return ask_cursor_agent(question)

# Example usage:
answer = quick_ask("what are the main components of this project?")
print(answer)